# Addition of SID History to Active Directory Object
An attacker can use the SID history attribute to gain additional privileges.

## Rule Content
```
- title: Addition of SID History to Active Directory Object
  id: 2632954e-db1c-49cb-9936-67d1ef1d17d2
  status: stable
  description: An attacker can use the SID history attribute to gain additional privileges.
  references:
  - https://adsecurity.org/?p=1772
  author: Thomas Patzke, @atc_project (improvements)
  tags:
  - attack.persistence
  - attack.privilege_escalation
  - attack.t1178
  logsource:
    product: windows
    service: security
    category: null
  detection:
    selection1:
      EventID:
      - 4765
      - 4766
    selection2:
      EventID: 4738
    selection3:
      SidHistory:
      - '-'
      - '%%1793'
    condition: selection1 or (selection2 and not selection3)
  falsepositives:
  - Migration of an account into a new domain
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:("4765" OR "4766") OR (event_id:"4738" AND (NOT (SidHistory:("\-" OR "%%1793")))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()